In [2]:
using DifferentialEquations, Plots, LinearAlgebra, LaTeXStrings
include("utils.jl")

animate_trajectory (generic function with 1 method)

# Geodésicas en Kerr

## Plano ecuatorial

Por el momento sólo tomemos en cuenta el plano ecuatorial $\theta = \pi/2$, en ese caso la métrica es

$$ ds^{2} = -\left( 1 - \dfrac{2M}{r} \right)dt^{2} - \dfrac{4aM}{r}dtd\varphi + \dfrac{r^{2}}{\Delta}dr^{2} + \left( r^{2} + a^{2} + \dfrac{2Ma^{2}}{r} \right)d\varphi^{2}. $$

Las órbitas están en términos de las cantidades conservadas: la energía $\varepsilon$ y el momento angular $\ell$ (a lo largo del eje de simetría) de la partícula, las cuales surgen de la independencia de $t$ y $\varphi$ de la métrica. En términos de los vectores de Killing $\boldsymbol{\xi}$, $\boldsymbol{\eta}$ y la 4-velocidad $\boldsymbol{u}$ se tiene que 

\begin{align*}
\varepsilon &= - \boldsymbol{\xi}\cdot\boldsymbol{u}, \\
\ell &= \boldsymbol{\eta}\cdot\boldsymbol{u}.
\end{align*}

Las ecuaciones de la geodésica para el caso de Schwarzschild son

\begin{align}
\dfrac{d^2 t}{d\lambda^2} &= \dfrac{2M}{2Mr - r^2} \left( \dfrac{dt}{d\lambda} \right) \left( \dfrac{dr}{d\lambda} \right), \\
\dfrac{d^2 r}{d\lambda^2} &= \dfrac{M(2M-r)}{r^3} \left( \dfrac{dt}{d\lambda} \right)^{2} - \dfrac{M}{2Mr - r^2} \left( \dfrac{dr}{d\lambda} \right)^{2} - (2M - r) \left( \dfrac{d\theta}{d\lambda} \right)^{2} - (2M - r)\sin^{2}\theta \left( \dfrac{d\phi}{d\lambda} \right)^{2}, \\
\dfrac{d^2 \theta}{d\lambda^2} &= - \dfrac{2}{r} \left( \dfrac{dr}{d\lambda} \right) \left( \dfrac{d\theta}{d\lambda} \right) + \cos\theta\sin\theta \left( \dfrac{d\phi}{d\lambda} \right)^{2}, \\
\dfrac{d^2 \phi}{d\lambda^2} &= -\dfrac{2}{r} \left( \left( \dfrac{dr}{d\lambda} \right) + r\cot\theta \left( \dfrac{d\theta}{d\lambda} \right) \right) \left( \dfrac{d\phi}{d\lambda} \right).
\end{align}

Tomando $\theta = \pi / 2$ y considerando las cantidades conservadas $\ell = r^{2}\sin^{2}\theta \frac{d\phi}{d\lambda}$, $\epsilon = \left(1-\frac{2M}{r}\right)\frac{dt}{d\lambda}$ las ecuaciones para $r$ y $\phi$ quedan

\begin{align}
\dfrac{d^2 r}{d\lambda^2} &= -\dfrac{M\epsilon^2}{r^2}\left( 1 - \dfrac{2M}{r}\right)^{-1} - \dfrac{M}{2Mr - r^2} \left( \dfrac{dr}{d\lambda} \right)^{2} + \left( 1 - \dfrac{2M}{r}\right)\dfrac{\ell^{2}}{r^3},\\
\dfrac{d^2 \phi}{d\lambda^2} &= -\dfrac{2}{r}\dfrac{dr}{d\lambda}\dfrac{d\phi}{d\lambda}.
\end{align}

O usando $\ell$ en la ecuación de $\phi$

\begin{align}
\dfrac{d^2 r}{d\lambda^2} &= -\dfrac{M\epsilon^2}{r^2}\left( 1 - \dfrac{2M}{r}\right)^{-1} - \dfrac{M}{2Mr - r^2} \left( \dfrac{dr}{d\lambda} \right)^{2} + \left( 1 - \dfrac{2M}{r}\right)\dfrac{\ell^{2}}{r^3},\\
\dfrac{d\phi}{d\lambda} &= \dfrac{\ell}{r^2}.
\end{align}

In [45]:
function Δ(r,p)
    M, a, ϵ, b, ℓ = p
    return r^2 - 2*M*r + a^2
end

function ρ²(r, θ, p)
    M, a, ϵ, b, ℓ = p
    return r^2 + (a*cos(θ))^2
end

function Σ²(r, θ, p)
    M, a, ϵ, b, ℓ = p
    return (r^2 + a^2)^2 - a^2*Δ(r,p)*(sin(θ))^2
end

function dt_dλ(r, θ, p)
    M, a, ϵ, b, ℓ = p
    return (Σ²(r, θ, p)*ϵ - 2*a*M*r*ℓ)/(Δ(r,p)*ρ²(r, θ, p))
end
    
function dϕ_dλ(r, θ, p)
    M, a, ϵ, b, ℓ = p
    return (2*a*M*r*ϵ + (ρ²(r, θ, p) - 2*M*r)*ℓ*(csc(θ))^2)/(Δ(r,p)*ρ²(r, θ, p))
end

function dθ_dλ(r, r′, θ, p)
    M, a, ϵ, b, ℓ = p    
    K = (((r^2 + a^2)*ϵ - a*ℓ)^2 - ρ²(r,θ,p)^2*r′^2)/Δ(r,p)
    
    return abs(sqrt(Complex((K - (ℓ - a*ϵ)^2 - (-a^2*ϵ^2 + ℓ^2*csc(θ)^2)*cos(θ)^2)/(ρ²(r,θ,p)^2))))    
end

dθ_dλ (generic function with 1 method)

In [46]:
function dr′_dλ(r, r′, θ, θ′, p)
    M, a, ϵ, b, ℓ = p
    
    num = (ρ²(r, θ, p)^2 * (r*(-a^2 + M*r) + a^2*(-M + r)*cos(θ)^2) * r′^2)/(a^2 + r*(-2*M + r)) + 2*a^2*cos(θ)*ρ²(r, θ, p)^2*sin(θ)*r′*θ′ +
          (a^2 + r*(-2*M + r))*(M*(-r^2 + a^2*cos(θ)^2)*dt_dλ(r,θ,p)^2 + r*ρ²(r, θ, p)^2*(θ′^2) + 2*a*M*(r^2 - a^2*cos(θ)^2)*sin(θ)^2*dt_dλ(r,θ,p)*dϕ_dλ(r,θ,p) + sin(θ)^2*(r*ρ²(r, θ, p)^2 + a^2*M*(-r + a*cos(θ))*(r + a*cos(θ))*sin(θ)^2)*dϕ_dλ(r,θ,p)^2 )
    den = ρ²(r, θ, p)^3
    
    return num/den
    
end

dr′_dλ (generic function with 1 method)

In [47]:
function dθ′_dλ(r, r′, θ, θ′, p)
    M, a, ϵ, b, ℓ = p
    
    num = -(a^2*cos(θ)*ρ²(r,θ,p)^2*sin(θ)*r′^2)/(a^2 + r*(-2*M + r)) + a^2*M*r*sin(2*θ)*dt_dλ(r,θ,p)^2 - 2*r*ρ²(r,θ,p)^2*r′*θ′ - 2*a*M*r*(a^2 + r^2)*sin(2*θ)*dt_dλ(r,θ,p)*dϕ_dλ(r,θ,p) + 
          cos(θ)*sin(θ)*(a^2*ρ²(r,θ,p)^2*θ′^2 + ((a^2 + r^2)*ρ²(r,θ,p)^2 + a^2*M*r*(3*a^2 + 4*r^2 + a^2*cos(2*θ))*sin(θ)^2)*dϕ_dλ(r, θ, p)^2 )
    den = ρ²(r, θ, p)^3
    
    return num/den
    
end

dθ′_dλ (generic function with 1 method)

In [48]:
function parameterized_kerr!(du, u, p, t)
    
    """
    u es un vector con las variables r'=dr/dλ, r, ϕ; p es un vector con los parámetros del agujero
    """

    r′, r, θ′, θ, ϕ = u
    M, a, ϵ, b, ℓ = p
    σ = sign(ℓ)
        
    du[1] = dr′ = dr′_dλ(r, r′, θ, θ′, p)
    du[2] = dr = r′
    du[3] = dθ′ = dθ′_dλ(r, r′, θ, θ′, p)
    du[4] = dθ = θ′
    du[5] = dϕ = dϕ_dλ(r, θ, p)
    
end

parameterized_kerr! (generic function with 1 method)

In [49]:
function intervalo_kerr(r, r′, ϕ, θ, θ′, p)
    M, a, ϵ, b, ℓ = p
    ds = -(1 - 2*M*r/ρ²(r, θ, p))*dt_dλ(r,θ,p)^2 - (4*M*a/r)*dϕ_dλ(r, θ, p)*dt_dλ(r,θ,p) + (r^2/Δ(r,p))*r′^2 + (r^2 + a^2 + 2*M*a^2/r)*dϕ_dλ(r, θ, p)^2
    return ds    
end

intervalo_kerr (generic function with 1 method)

In [51]:
function geodesics_kerr(p, r0, r′0, ϕ0, θ0, λ_final, dλ_min, dλ_max=2*dλ_min)    
    M, a, ϵ, b, ℓ = p
    
    if a > M
        println("Error: a > M")
        return 
    end
    
    Δ0 = Δ(r0, p)
    dt0_dλ = dt_dλ(r0, θ0, p)
    dϕ0_dλ = dϕ_dλ(r0, θ0, p)
    dθ0_dλ = dθ_dλ(r0, r′0, θ0, p)
    
    θ′0 = dθ_dλ(r0, r′0, θ0, p)
    tspan = (0.0, λ_final)
    
    u0 = [r′0, r0, θ′0, θ0, ϕ0]  
    
    prob = ODEProblem(parameterized_kerr!, u0, tspan, p)
    sol = solve(prob, dtmin=dλ_min, dtmax=dλ_max, force_dtmin=true);
       
    r′s, rs, θ′s, θs, ϕs = sol[1,:], sol[2,:], sol[3,:], sol[4,:], sol[5,:]
    
    xs = [rs[i]*cos(ϕs[i]) for i in 1:length(sol)]   
    ys = [rs[i]*sin(ϕs[i]) for i in 1:length(sol)] 
    ds = [-(1 - 2*M/rs[i])*(dt_dλ(rs[i], θs[2], p))^2 - (4*a*M/rs[i])*dt_dλ(rs[i], θs[i], p)*dϕ_dλ(rs[i], p) + 
          (rs[i]^2/Δ(rs[i],p))*(r′s[i])^2 + (rs[i]^2 + a^2 + 2*M*a^2/rs[i])*(dϕ_dλ(rs[i], p))^2 for i in 1:length(rs)]
    
    return xs, ys, ds
    
end

geodesics_kerr (generic function with 2 methods)

Parece que hay un error en alguna de las ecuaciones, pero no es sencillo ver en qué ecuación está el error.

In [59]:
M = 10
a = 9.9
ℓ = 1
n = 1 # número de vueltas
b = 3*sqrt(3)*M + 3.4823*M*exp(-2*n*π)
ϵ = ℓ/b

p = [M, a, ϵ, b, ℓ]

r0 = 5*M
r′0 = 5
ϕ0 = π/2
θ0 = 0.0

λ_final = 10.0
dλ = 1e-6

@time xs, ys, ds = geodesics_kerr(p, r0, r′0, ϕ0, θ0, λ_final, dλ);

animate_trajectory(xs, ys)

┌ Warning: First function call produced NaNs. Exiting.
└ @ OrdinaryDiffEq /home/david/.julia/packages/OrdinaryDiffEq/VPJBD/src/initdt.jl:76
┌ Warning: Automatic dt set the starting dt as NaN, causing instability.
└ @ OrdinaryDiffEq /home/david/.julia/packages/OrdinaryDiffEq/VPJBD/src/solve.jl:459
┌ Warning: NaN dt detected. Likely a NaN value in the state, parameters, or derivative value caused this outcome.
└ @ DiffEqBase /home/david/.julia/packages/DiffEqBase/V7P18/src/integrator_interface.jl:323


BoundsError: BoundsError: attempt to access 1-element Array{Float64,1} at index [2]